In [13]:
%reset -f


import numpy as np
import qutip as qt
import networkx as nx
import matplotlib.pyplot as plt
import scipy

import itertools
import copy
import time
import os

from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit import ParameterVector


from qiskit_ibm_runtime import Estimator as RuntimeEstimator, QiskitRuntimeService, Options, Session
from qiskit import QuantumCircuit, transpile, Aer, execute


print("Cell evaluated")



Cell evaluated


# Scale factor

Recall that for an operator $\hat{M}$, the scale factor is given as follows:

\begin{align*}
\mathcal{f} = \frac{\langle \hat{M} \rangle _{\textrm{noiseless}}}{\langle \hat{M} \rangle _{\textrm{hardware}}}
\end{align*}

for the purposes of rescaling our measured ground state energy:

\begin{align*}
E_g = \mathcal{f} \langle 0 | U^\dagger\left( \theta \right) \hat{H} U\left( \theta \right) | 0 \rangle
\end{align*}

Our choie of $\hat{M}$ are the following Hamiltonians:

\begin{align*}
\hat{\mathcal{G}}_1 = \sum_{\langle i, j \rangle \in \mathcal{G}_1} \left( X_i X_j + Y_i Y_j + Z_i Z_j \right) 
\text{ and } 
\hat{\mathcal{G}}_2 = \sum_{\langle i, j \rangle \in \mathcal{G}_2} \left( X_i X_j + Y_i Y_j + Z_i Z_j \right)
\end{align*}

where $\mathcal{G}_1$ and $\mathcal{G}_2$ are the two perfect matchings of `ibmq_guadalupe`'s inner ring of qubits. 


Here we have already optimized our ansatz to minimize $\langle \hat{\mathcal{G}}_1 \rangle$ and $\langle \hat{\mathcal{G}}_2 \rangle$. This notebook uses the saved parameters to find the scale factor using the `ibmq_guadalupe` backend.



In [14]:
def cryansatz(nqubits, layers, init_bitstring): # Hardware efficient ansatz that is spin preserving.
    
    # Separate physical and "virtual" qubits (i.e. qubits used) 
    # to prevent errors about how the number of qubits don't match.
    # However, gates will only act on the number of used qubits set by the "nqubits" argument.
    
    # From here, we will use the "initial_layout" argument during transpile to map the extra qubits
    # in the circuit to the physical ones we want to avoid (i.e. 0,6,9,15)
    physical_qubits = 16
    
    assert nqubits%2 == 0
    assert nqubits <= physical_qubits
    
    
    my_circ = QuantumCircuit(physical_qubits)
    nparams = int((nqubits/2) * layers)
    
    params = ParameterVector('θ', nparams)
    params_idx = 0
    
    for x_ in range(len(init_bitstring)):
        if init_bitstring[x_] == "1":
            my_circ.x(x_)
            
    my_circ.barrier()
    
    for l_ in range(layers):
        if l_%2 ==0:
            for q2_ in range(0, nqubits,2):
                if q2_+1 == nqubits:
                    qb1 = q2_
                    qb2 = 0
                else:
                    qb1 = q2_
                    qb2 = q2_ + 1
                    
                my_circ.cx(qb1, qb2)
                my_circ.cry(params[params_idx], qb2, qb1)
                params_idx += 1
                my_circ.cx(qb1, qb2)
                my_circ.rzz(np.pi/2, qb2, qb1)
        else:
            for q2_ in range(1, nqubits,2):
                if q2_+1 == nqubits:
                    qb1 = q2_
                    qb2 = 0
                else:
                    qb1 = q2_
                    qb2 = q2_ + 1
                    
                my_circ.cx(qb1, qb2)
                my_circ.cry(params[params_idx], qb2, qb1)
                params_idx += 1
                my_circ.cx(qb1, qb2)
                my_circ.rzz(np.pi/2, qb2, qb1)

                
        my_circ.barrier()
                
    return my_circ

print("Cell evaluated")


Cell evaluated


# Rings and Perfect Matchings

Finding $\langle \hat{\mathcal{G}}_1 \rangle$ and $\langle \hat{\mathcal{G}}_2 \rangle$ was done by optimizing over a single Hamiltonian, $\hat{\mathcal{G}}$. Here, we use Qiskit's transpile functionality and construct the Hamiltonians based on which perfect matching, $\mathcal{G}_1$ or $\mathcal{G}_2$, we want to act on.

`guadalupe_ring_1` and `guadalupe_ring_2` are the edges for the perfect matching graphs $\mathcal{G}_1$ and $\mathcal{G}_2$ respectively.


In [15]:
from qiskit.providers.fake_provider import FakeGuadalupeV2
fake_backend = FakeGuadalupeV2() 

# Both perfect matchings for the inner ring of guadalupe device.
# Hard coded for now since we are only using guadalupe and other devices have different topo.
guadalupe_ring_1 = [[2,3],[5,8],[11,14],[13,12],[10,7],[4,1]]
guadalupe_ring_2 = [[1,2],[3,5],[8,11],[14,13],[12,10],[7,4]]


# Layout maps the qubits to the connections above.
initial_layout = [2,3,5,8,11,14,13,12,10,7,4,1,0,6,9,15]
initial_layout2 = [1,2,3,5,8,11,14,13,12,10,7,4,0,6,9,15]


draw_circ = cryansatz(12,4,'101010101010')


print("Depth:", draw_circ.depth())
print("Gate count:", sum(draw_circ.count_ops().values()))
print("Nonlocal gate count:", draw_circ.num_nonlocal_gates())

# display(draw_circ.draw(fold=5000))

print("Cell evaluated")



Depth: 17
Gate count: 107
Nonlocal gate count: 96
Cell evaluated


### Constructing our Hamiltonians

Constructing $\hat{\mathcal{G}}_1$ and $\hat{\mathcal{G}}_2$ based on the perfect matchings above.

In [16]:
gjkl2 = time.time()

print("Preparing the Hamiltonians...")

ring1_hmin = []
ring2_hmin = []


ring1_obs = [] 
for conn in guadalupe_ring_1:
    ring1_obs.append(("XX", conn, 1))
    ring1_obs.append(("YY", conn, 1))
    ring1_obs.append(("ZZ", conn, 1))
    
    
ring2_obs = [] 
for conn in guadalupe_ring_2:
    ring2_obs.append(("XX", conn, 1))
    ring2_obs.append(("YY", conn, 1))
    ring2_obs.append(("ZZ", conn, 1))
    
    
# Pads to 16 qubits for running on ibmq_guadalupe
ring1_obs = SparsePauliOp.from_sparse_list(ring1_obs,num_qubits = 16)
ring2_obs = SparsePauliOp.from_sparse_list(ring2_obs,num_qubits = 16)


print("Cell evaluated")

Preparing the Hamiltonians...
Cell evaluated


### Loading our saved data....

Inside load_saved_dat is a list of runs that minimizes $\langle \hat{\mathcal{G}} \rangle$. Each run is a dictionary with the following keys:
- `seed`: Integer that seeds the optimizer
- `opt params`: Optimal parameters for our circuit
- `cflist`: Cost function values after each optimizer iteration
- `paramlist`: Theta values after each optimizer iteration

Loading the optimal parameters into a noise-less backend will give a value close to -18.

In [17]:
load_data = np.load("TrivH-cryansatz-12l4-notranspile-Rotosolvedat.npy", allow_pickle = True)

# Dictionary of seed:parameters
opt_param_d = {} 

for runn in load_data:
    opt_param_d[runn['seed']] = runn['opt params']
    
print("Number of parameters to run:",  len(load_data))


Number of parameters to run: 15


# Finding the scale factor for ibmq_guadalupe using Runtime

Due to the highly stochastic nature of quantum experiments, multiple optimization runs to find the minimum of $\langle \hat{\mathcal{G}} \rangle$ are used to obtain the scale factor $\mathcal{f}$.

Each run produces a single scale factor value $\mathcal{f}_i$, and $\mathcal{\vec{f}} = \{ {\mathcal{f}_1, ... , \mathcal{f}_I} \}$ after calculating the scale factor for all runs. Similarly, the multiple VQE runs optimizing the ground state energy of the Kagome Hamiltonian produce a list of $\langle \hat{H}_{\textrm{kagome}} \rangle _j$ values $\{ \langle \hat{H}_{\textrm{kagome}}  \rangle _1 , ..., \langle \hat{H}_{\textrm{kagome}}  \rangle _J\}$


The outer product of these two lists will yield a matrix of approximated ground state energies for the Kagome Hamiltonian:


\begin{align*}
E_{\textrm{kagome}}^{ij} = \mathcal{f}_i \langle \hat{H}_{\textrm{kagome}}  \rangle_j
\end{align*}



The blocks of code below runs the saved parameters on `ibmq_guadalupe` to obtain $\mathcal{\vec{f}}$, and saves it to disk.




In [18]:
print("Loading runtime service.... Est wait: 10s")
ttg1 =time.time()
service = QiskitRuntimeService(channel="ibm_quantum",
                               # instance='ibm-q-community/ibmquantumawards/open-science-22'
                               # instance='ibm-q/open/main'
                              ) 

runtime_backend = 'ibmq_guadalupe'
print("Service loaded. Time taken: %.2fs" %(time.time() - ttg1))


Loading runtime service.... Est wait: 10s
Service loaded. Time taken: 229.53s


# Sending the jobs via a runtime session

It is important to fix the options below during the session, as the same options need to be used later when using `ibmq_guadalupe` to find $\langle \hat{H}_{\textrm{kagome}} \rangle$.

Here, we leverage Runtime's ability to send multiple jobs at once to be queued. The first loop is used to send the jobs to the backend. Once all the jobs has been sent, a second loop queries the results and saves them to a list. This saves some overhead as subsequent jobs can be sent without having to wait for earlier ones to finish. 

We have also set the total number of shots to the maximum to for our scale factor to capture the noise profile of `ibmq_guadalupe` as accurately as possible.


In [20]:
options = Options()                   # https://qiskit.org/documentation/partners/qiskit_ibm_runtime/stubs/qiskit_ibm_runtime.options.Options.html
options.optimization_level = 3        # 3 is Default, https://qiskit.org/documentation/partners/qiskit_ibm_runtime/how_to/error-suppression.html
options.resilience_level = 2          # 0: none, 1: readout errors (TREX), 2: ZNE, 3: PEC
options.max_execution_time = None
options.execution.shots = 100000      # 4000 is default
options.execution.init_qubits = True  # Reset Qubits after each shot
options.transpilation.skip_transpilation = True # Don't transpile using qiskit_ibm_runtime.Options... EVER

gjkl1 = time.time()

    
with Session(service=service, backend=runtime_backend) as session:
    
    rt_estimator = RuntimeEstimator(session=session, options=options)

    job1_list = []           # For saving jobs sent to measure <G1>
    job1_id_list = []        # For saving jobs IDs sent to measure <G1>
    seedlist1 = []
    
    print(f">>> Running circuits... ")
    
    for seed, params in opt_param_d.items():
        
        mycirc = cryansatz(12,4,'101010101010')
        # Bind first before transpile, because optimization for these runs was done on non-transpiled circuit.
        statecirc = mycirc.bind_parameters(params) 


        transpiled_circ1 = transpile(statecirc,                   # Transpiled circuit to map qubits to G1
                            backend = fake_backend,
                            optimization_level = 3,
                            initial_layout = initial_layout,
                           )


        job1 = rt_estimator.run([transpiled_circ1], [ring1_obs])  # Circuit and obs for the 1st perfect matching
        job1_list.append(job1)
        job1_id_list.append(job1.job_id())

        sess1_id = session.session_id
        seedlist1.append(seed)
        print(f">>> Stpt: {seed}, Job1 ID: {job1.job_id()}, Status: {job1.status()}")
        
        
    session.close()
    
print()
print(f">>> All jobs sent for ring1! Time taken: {(time.time() - gjkl1)}s. Sending ring2 jobs....")
print()

gjkl1 = time.time()
with Session(service=service, backend=runtime_backend) as session:
    
    rt_estimator = RuntimeEstimator(session=session, options=options)
    
    job2_list = []           # For saving jobs sent to measure <G2>
    job2_id_list = []        # For saving jobs IDs sent to measure <G2>
    seedlist2 = []
    
    print(f">>> Running circuits... ")
    
    for seed, params in opt_param_d.items():
        
        mycirc = cryansatz(12,4,'101010101010')
        # Bind first before transpile, because optimization for these runs was done on non-transpiled circuit.
        statecirc = mycirc.bind_parameters(params) 


        transpiled_circ2 = transpile(statecirc,                   # Transpiled circuit to map qubits to G2
                            backend = fake_backend,
                            optimization_level = 3,
                            initial_layout = initial_layout2, 
                           )


        job2 = rt_estimator.run([transpiled_circ2], [ring2_obs])  # Circuit and obs for 2nd perfect matching
        job2_list.append(job2)
        job2_id_list.append(job2.job_id())
        
        sess2_id = session.session_id
        seedlist2.append(seed)
        print(f">>> Stpt: {seed}, Job2 ID: {job2.job_id()}, Status: {job2.status()}")
    session.close()
    
    
print()
print(f">>> All jobs sent for ring2! Time taken: {(time.time() - gjkl1)}s. Awaiting results....")
print(f"Session IDs: {sess1_id, sess2_id}")
print()





>>> Running circuits... 
>>> Stpt: 75970, Job1 ID: cgsq2qsqunt0rafs97fg, Status: JobStatus.QUEUED
>>> Stpt: 27940, Job1 ID: cgsq2r2j96cnav4vf1j0, Status: JobStatus.QUEUED
>>> Stpt: 87840, Job1 ID: cgsq2rkqunt0rafs991g, Status: JobStatus.QUEUED
>>> Stpt: 73533, Job1 ID: cgsq2s02tle0vokd6vmg, Status: JobStatus.QUEUED
>>> Stpt: 43508, Job1 ID: cgsq2skqunt0rafs9an0, Status: JobStatus.QUEUED
>>> Stpt: 82037, Job1 ID: cgsq2tdc26uins2mp5h0, Status: JobStatus.QUEUED
>>> Stpt: 47977, Job1 ID: cgsq2tkqunt0rafs9ci0, Status: JobStatus.QUEUED
>>> Stpt: 72857, Job1 ID: cgsq2to2tle0vokd7320, Status: JobStatus.QUEUED
>>> Stpt: 96697, Job1 ID: cgsq2ub7iuhj2v48ak90, Status: JobStatus.QUEUED
>>> Stpt: 44508, Job1 ID: cgsq2uqj96cnav4vf9ig, Status: JobStatus.QUEUED
>>> Stpt: 36766, Job1 ID: cgsq2vaj96cnav4vfa50, Status: JobStatus.QUEUED
>>> Stpt: 25872, Job1 ID: cgsq30ij96cnav4vfcgg, Status: JobStatus.QUEUED
>>> Stpt: 41836, Job1 ID: cgsq30tevgbv2i5sq6lg, Status: JobStatus.QUEUED
>>> Stpt: 39185, Job1 ID: 

### This part here assumes all jobs were sent and ran successfully

The jobs are sent in the order of where they are in the saved file, and are retrieved chronologically when querying `service.jobs`, so manually keeping track of them is not necessary, although still possible based on the seedlist saved when running the cell above. 

However, this allows for jobs to be retrieved that were not run during this session by providing the session_id.

In [ ]:
session_1_id_to_query = sess1_id
session_2_id_to_query = sess2_id

# session_1_id_to_query = "cgsq2qsqunt0rafs97fg"                 # If we want to load existing session IDs
# session_2_id_to_query = "cgsq3237iuhj2v48aq70"                 # If we want to load existing session IDs

print("Loading job services...")
job_1_list = service.jobs(limit=None, session_id=session_1_id_to_query ,descending=True)
job_2_list = service.jobs(limit=None, session_id=session_2_id_to_query ,descending=True)

njobs = len(job1_list)

print("Retrieving job results...")
tkfg1 = time.time()

hbknd1_list = []         # For saving measured <G1> values
hbknd2_list = []         # For saving measured <G2> values

for idx in range(njobs):
    
    # Print statuses first in case job is not ready to pull the result.
    print(f"Job1 ID: {job_1_list[idx].job_id()}, Status: {job_1_list[idx].status()},",
          f"Job2 ID: {job_2_list[idx].job_id()}, Status: {job_2_list[idx].status()}"
         )
    
    ham_min_hardware1 = job_1_list[idx].result().values[0]     # Measured <G1> values
    ham_min_hardware2 = job_2_list[idx].result().values[0]     # Measured <G2> values

    hbknd1_list.append(ham_min_hardware1)
    hbknd2_list.append(ham_min_hardware2)


print(f">>> Exp values (backend): {hbknd1_list, hbknd2_list}, Time taken: {(time.time() - tkfg1):<5.3}s")

In [82]:
savedict = dict({})
savedict["shots"] = options.execution.shots
savedict["hbknd1"] = hbknd1_list
savedict["hbknd2"] = hbknd2_list
savedict["fval1"] = -18/np.array(hbknd1_list)                     # f-value from <G1>_noiseless/<G1>_hardware
savedict["fval2"] = -18/np.array(hbknd2_list)                     # f-value from <G2>_noiseless/<G2>_hardware
savedict["meanf"] = (savedict["fval1"] + savedict["fval2"])/2     # f-value of run is the average of two perfect matchings
savedict["sessid"] = [session_1_id_to_query, session_2_id_to_query]
savedict["stptlist"] = seedlist


filename = "ScaleFactorData-Guadalupe-"+str(options.execution.shots)+".npy"
np.save(filename, savedict)
print("Data saved")
